# Librerias

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import lightgbm as lgb
import os
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
import joblib
from catboost import CatBoostClassifier

# Preprocesado de Datos

In [ ]:
def agregar_lags(df, columna, lags):
    for lag in lags:
        df[f'lag_{lag}'] = df[columna].shift(lag)
    return df

def agregar_medias_moviles(df, columna, ventanas):
    for ventana in ventanas:
        df[f'media_movil_{ventana}'] = df[columna].shift(1).rolling(window=ventana, min_periods=1).mean()
    return df

def agregar_variables_ciclicas(df, columna_mes, columna_fecha):
    df['mes_sin'] = np.sin(2 * np.pi * df[columna_mes] / 12)
    df['mes_cos'] = np.cos(2 * np.pi * df[columna_mes] / 12)
    df['dia_semana_sin'] = np.sin(2 * np.pi * pd.to_datetime(df[columna_fecha]).dt.weekday / 7)
    df['dia_semana_cos'] = np.cos(2 * np.pi * pd.to_datetime(df[columna_fecha]).dt.weekday / 7)
    return df

def agregar_ewma(df, columna, spans):
    for span in spans:
        df[f'ewma_{span}'] = df[columna].shift(1).ewm(span=span, adjust=False).mean()
    return df

def transformacion_a_clasificacion(df, class_range):
    max = ((df_r['Accesos'].max())//class_range)+2
    bins = [0]
    labels = []
    for i in range(max):
        bins.append((i+1)*class_range)
        labels.append(i)
    df_r['Accesos'] = pd.cut(df_r['Accesos'], bins=bins, labels=labels, right=False)
    return df
def obtener_semestre(mes):
    if 3 <= mes <= 7:
        return 1  # Primer semestre
    elif 8 <= mes <= 12:
        return 2  # Segundo semestre
    else:
        return 0  # Fuera de semestre (enero, febrero)
    
def obtener_semana_semestre(fecha):
    mes = fecha.month
    dia = fecha.day
    if 3 <= mes <= 7:
        inicio_semestre = datetime(fecha.year, 3, 1)
    elif 8 <= mes <= 12:
        inicio_semestre = datetime(fecha.year, 8, 1)
    else:
        return 0  # Fuera de semestre
    delta = fecha - inicio_semestre
    semana_semestre = delta.days // 7 + 1
    return semana_semestre

In [ ]:
try:
    df = pd.read_csv("Datasets/accesos_biblioteca.csv")
    print("Dataset cargado desde 'Datasets/accesos_biblioteca.csv'")
except FileNotFoundError:
    print("ERROR: No se encontró el archivo 'accesos_biblioteca.csv'.")
    exit(1)

In [ ]:
for col in ['Accesos']:
    df[col] = df[col].fillna(0).astype(int)

df = df[
    pd.to_datetime(df['Fecha']).dt.weekday != 6
].reset_index(drop=True)

df = agregar_medias_moviles(df, 'Accesos', [7, 14, 30])
df = agregar_variables_ciclicas(df, 'Mes', 'Fecha')
df = agregar_ewma(df, 'Accesos', [7, 14, 30])

df['Semestre'] = df['Mes'].apply(obtener_semestre)
df['Semana_Semestre'] = pd.to_datetime(df['Fecha']).apply(obtener_semana_semestre)

df['Fecha'] = pd.to_datetime(df['Fecha'])

df = agregar_lags(df, 'Accesos', [7, 14, 21])
df = transformacion_a_clasificacion(df, 500)

df = df.dropna().reset_index(drop=True)
df.tail(5)

In [ ]:
X = df.drop(columns=['Fecha', 'Accesos'])
y = df['Accesos']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print(f"Datos de entrenamiento: {len(X_train)}")
print(f"Datos de prueba: {len(X_test)}")

# Visualizacion de Datos

In [ ]:
print(f"\n--- Preparación de Datos Finalizada ---")
print(f"Prophet: Datos de entrenamiento: {len(train_df)} | Datos de prueba: {len(test_df)}")

In [ ]:
data_prophet.tail(5)

In [ ]:
print(" Nuevas columnas creadas \n")

# Columnas originales del dataset base
columnas_originales = ['Fecha', 'Día', 'Mes', 'Año', 'Vacaciones', 'Accesos']

# Mostrar todas las columnas actuales
columnas_actuales = list(df.columns)
nuevas_columnas = [col for col in columnas_actuales if col not in columnas_originales]

print(" Dataset original - Columnas iniciales:")
print(f"   Columnas: {columnas_originales}")
print(f"   Total: {len(columnas_originales)} columnas\n")

print(" Nuevas columnas agregadas:")
for i, col in enumerate(nuevas_columnas, 1):
    print(f"   {i:2d}. {col}")

print(f" Resumen de columnas")
print(f" Columnas originales: {len(columnas_originales)}")
print(f" Nuevas columnas: {len(nuevas_columnas)}")
print(f" Total final: {len(columnas_actuales)} columnas")
print(f" Shape del dataset: {df.shape}")

print(f" Categorías de nuevas características:")
print("  Variables de tendencia: media_movil_7, media_movil_14, media_movil_30")
print("  Variables cíclicas: mes_sin, mes_cos, dia_semana_sin, dia_semana_cos") 
print("  Medias exponenciales: ewma_7, ewma_14, ewma_30")
print("  Variables académicas: Semestre, Semana_Semestre, Semana_Certamen")
print("  Variables lag: lag_7, lag_14, lag_21 (se agregarán después)")

# Mostrar una muestra del dataset con las nuevas columnas
print("\n Muestra del Dataset con nuevas columnas:")
print(df[['Fecha', 'Accesos'] + nuevas_columnas[:5]].head(3))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['Fecha'], df['Accesos'], linestyle='-', color='b', alpha=0.5)

# Puntos normales (no vacaciones ni sábados)
no_vac_no_sab = (df['Vacaciones?'] == 0) & (df['Fecha'].dt.weekday != 5)
plt.scatter(df['Fecha'][no_vac_no_sab], df['Accesos'][no_vac_no_sab], color='b', label='Día normal')

# Puntos de vacaciones
vac = (df['Vacaciones?'] == 1)
plt.scatter(df['Fecha'][vac], df['Accesos'][vac], color='r', label='Vacaciones?')

# Puntos de sábados
sab = (df['Vacaciones?'] == 0) & (df['Fecha'].dt.weekday == 5)
plt.scatter(df['Fecha'][sab], df['Accesos'][sab], color='g', label='Sábado')

plt.title('Accesos Diarios a la Universidad')
plt.xlabel('Fecha')
plt.ylabel('Número de Accesos')
plt.xticks(rotation=45)
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Matriz de coorrelación
plt.figure(figsize=(12, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Matriz de Correlación de Variables', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
print("Dataset final después de ingeneiría de características\n")

# Mostrar todas las columnas finales organizadas por categoría
print("Columnas organizadas por categoría:")

print("\n Variables originales:")
originales = ['Fecha', 'Día', 'Mes', 'Año', 'Vacaciones', 'Accesos']
for col in originales:
    if col in df.columns:
        print(f" {col}")

print("\n Variables de tendencia/suavizado:")
tendencia = [col for col in df.columns if 'media_movil' in col or 'ewma' in col]
for col in tendencia:
    print(f" {col}")

print("\n Variables cíclicas/temporales:")
ciclicas = [col for col in df.columns if any(x in col for x in ['sin', 'cos'])]
for col in ciclicas:
    print(f" {col}")

print("\n Variables del contexto académico:")
academicas = [col for col in df.columns if any(x in col for x in ['Semestre', 'Semana', 'Certamen'])]
for col in academicas:
    print(f" {col}")

print("\n Variables lag (retardos temporales):")
lags = [col for col in df.columns if 'lag_' in col]
for col in lags:
    print(f" {col}")

print(f"\n Resumen Final:")
print(f" Total de columnas: {len(df.columns)}")
print(f" Filas en el dataset: {len(df)}")
print(f" Nuevas características creadas: {len(df.columns) - 6}")  # 6 son las originales

# Funciones Utiles

In [ ]:
def graph_pred(Y_pred, Model):
    results_df = pd.DataFrame({
        'Índice': X_test.index,
        'Real': y_test,
        'Predicción': Y_pred
    }).sort_values(by='Índice')

    plt.figure(figsize=(12, 6))
    plt.plot(results_df['Índice'], results_df['Real'], label='Real', color='blue', linewidth=2)
    plt.plot(results_df['Índice'], results_df['Predicción'], label='Predicción', color='green', linestyle='--', linewidth=2)
    plt.title('Resultados de Predicción', fontsize=14)
    plt.xlabel('Índice', fontsize=12)
    plt.ylabel('Número de Accesos', fontsize=12)
    plt.xticks(rotation=45)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.5)
    plt.tight_layout()
    plt.show()
    plt.savefig(f'{Model}_predicciones.png')

# Random Forest

In [ ]:
os.makedirs('Archivos Auxiliares', exist_ok=True)
if "modelo_RandomForest.pkl" not in os.listdir('Archivos Auxiliares'):
    tscv = TimeSeriesSplit(n_splits=5)
    param_grid = {
        'n_estimators': [100, 300, 500, 1000],
        'max_depth': [3, 5, 8, None],
        'min_samples_split': [2, 5, 8, 10],
        'min_samples_leaf': [1, 2, 4, 8],
        'max_features': [None, 'sqrt', 'log2', 0.8, 0.4]
    }
    grid_search = GridSearchCV(
        RandomForestClassifier(random_state=42, class_weight='balanced'),
        param_grid,
        cv=tscv,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=2 # Aumentamos el verbose para seguir mejor el progreso
    )
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    model = RandomForestClassifier(**best_params, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)
    joblib.dump(model, 'Archivos Auxiliares/modelo_RandomForest.pkl')
    print("Modelo guardado como 'Archivos Auxiliares/modelo_RandomForest.pkl'")
else:
    model = joblib.load('Archivos Auxiliares/modelo_RandomForest.pkl')
    print("Modelo cargado desde 'Archivos Auxiliares/modelo_RandomForest.pkl'")

In [ ]:
# Importancia de las características
importancias = model.feature_importances_
indices = np.argsort(importancias)[::-1]
print("\n Importancia de las características \n")
for i in range(X.shape[1]):
    print(f"{i + 1:2d}. {X.columns[indices[i]]}: {importancias[indices[i]]:.4f}")

In [ ]:
# Eliminar características con importancia menor a 0.01
umbral_importancia = 0.01
caracteristicas_seleccionadas = X.columns[importancias >= umbral_importancia]
X_train = X_train[caracteristicas_seleccionadas]
X_test = X_test[caracteristicas_seleccionadas]

#Reentrenar el modelo con las características seleccionadas
model.fit(X_train, y_train)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
graph_pred(Y_pred, 'RandomForest_C')

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(16, 8)) 

plt.plot(
    df['Fecha'],
    df['Accesos'],
    label='Datos Reales (Observados)',
    color='#1f77b4',
    linewidth=2.5,
    alpha=0.8
)

plt.plot(
    df['Fecha'].loc[X_test.index],
    Y_pred,
    label='Predicción (Test)',
    color='#d62728',
    linestyle='--',
    linewidth=2.5,
    marker='o',
    markersize=5,
    alpha=0.9
)

plt.xlabel('Fecha', fontsize=14, labelpad=10)
plt.ylabel('Número de Accesos', fontsize=14, labelpad=10)
plt.title(
    'Rendimiento del Modelo: Predicciones vs. Valores Reales',
    fontsize=18,
    fontweight='bold',
    pad=20
)

plt.xticks(rotation=45, ha='right', fontsize=11)
plt.yticks(fontsize=11)

# Leyenda y Rejilla
plt.legend(loc='upper left', fontsize=12, frameon=True, shadow=True)
plt.grid(True, axis='both', linestyle='-', alpha=0.5)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

f1_score(y_test, Y_pred, average='weighted')
print(classification_report(y_test, Y_pred))
print(confusion_matrix(y_test, Y_pred))
ConfusionMatrixDisplay.from_predictions(y_test, Y_pred, cmap=plt.cm.Blues)

# LightGBM

In [ ]:
os.makedirs('Archivos Auxiliares', exist_ok=True)
if "modelo_LightGBM.pkl" not in os.listdir('Archivos Auxiliares'):
    NUM_CLASS = df['Accesos'].max()+1
    tscv = TimeSeriesSplit(n_splits=5)
    param_grid = {
        'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2],
        'n_estimators': [100, 300, 500, 1000, 2000],
        'num_leaves': [16, 32, 64, 128],
        # opcional (incluye solo si sospechas overfit)
        'max_depth': [5, 8, 10, 15],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
    }

    print("\nIniciando entrenamiento y ajuste de hiperparámetros de LightGBM...")

    # 1. Búsqueda de Hyperparámetros
    grid_search = GridSearchCV(
        lgb.LGBMClassifier(num_class=NUM_CLASS,random_state=42, n_jobs=-1, objective='multiclass'),
        param_grid,  # Usando el conjunto de parámetros mejorado
        cv=tscv,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=2 # Aumentamos el verbose para seguir mejor el progreso
    )

    grid_search.fit(X_train, y_train)

    print(f'Mejores parámetros para LightGBM: {grid_search.best_params_}')

    # 2. Entrenamiento del modelo final con los mejores parámetros
    best_params = grid_search.best_params_
    model = lgb.LGBMClassifier(**best_params, num_class=NUM_CLASS, random_state=42, n_jobs=-1, objective='multiclass', metric='multi_logloss')
    model.fit(X_train, y_train)
    joblib.dump(model, 'Archivos Auxiliares/modelo_LightGBM.pkl')
    print("Modelo LightGBM guardado como 'Archivos Auxiliares/modelo_LightGBM.pkl'")
# Guardar el modelo
else:
    model = joblib.load('Archivos Auxiliares/modelo_LightGBM.pkl')
    print("Modelo LightGBM cargado desde 'Archivos Auxiliares/modelo_LightGBM.pkl'")

In [ ]:
# Importancia de las características
importancias = model.feature_importances_
indices = np.argsort(importancias)[::-1]
print("\nImportancia de las características\n")
for i, idx in enumerate(indices):
    if idx < len(X.columns):
        print(f"{i + 1:2d}. {X.columns[idx]}: {importancias[idx]:.4f}")

In [ ]:
# Eliminar características con importancia menor a 50
umbral_importancia = 50

# Solo considera las columnas usadas por el modelo y presentes en X_train/X_test
columnas_modelo = X.columns[:len(importancias)]
FEATURES = [col for col, imp in zip(columnas_modelo, importancias) 
            if imp >= umbral_importancia and col in X_train.columns and col in X_test.columns]

X_train = X_train[FEATURES]
X_test = X_test[FEATURES]

# Reentrenar el modelo con las características seleccionadas
model.fit(X_train, y_train)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
graph_pred(Y_pred, 'LightGBM_C')

In [ ]:
# Graficar importancia de variables
importances = model.feature_importances_
importance_df = pd.DataFrame({"Feature": FEATURES, "Importance": importances})
importance_df = importance_df.sort_values(by="Importance", ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(importance_df["Feature"], importance_df["Importance"], color='skyblue')
plt.xlabel("Importancia")
plt.ylabel("Feature")
plt.title("LightGBM: Importancia de variables")
plt.gca().invert_yaxis() 
plt.tight_layout()
plt.savefig('lightgbm_importancia.png')

print("\nSe han generado las siguientes visualizaciones:")
print("- lightgbm_predicciones.png (Comparación de valores reales vs. predichos)")
print("- lightgbm_importancia.png (Importancia de las variables)")

# CatBoost

In [ ]:
os.makedirs('Archivos Auxiliares', exist_ok=True)
if "modelo_CatBoost.pkl" not in os.listdir('Archivos Auxiliares'):
    tscv = TimeSeriesSplit(n_splits=5)
    param_grid = {
        'iterations': [100, 300, 500, 1000],
        'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [1, 3, 5],
    }

    print("\nIniciando entrenamiento y ajuste de hiperparámetros de CatBoost...")

    # 1. Búsqueda de Hyperparámetros
    grid_search = GridSearchCV(
        CatBoostClassifier(loss_function='MultiClass', eval_metric='MultiClass', random_state=42, verbose=0),
        param_grid,  # Usando el conjunto de parámetros mejorado
        cv=tscv,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=2 # Aumentamos el verbose para seguir mejor el progreso
    )

    grid_search.fit(X_train, y_train)

    print(f'Mejores parámetros para CatBoost: {grid_search.best_params_}')

    # 2. Entrenamiento del modelo final con los mejores parámetros
    best_params = grid_search.best_params_
    model = CatBoostClassifier(**best_params, loss_function='MultiClass', eval_metric='MultiClass', random_state=42, verbose=0)
    model.fit(X_train, y_train)
    joblib.dump(model, 'Archivos Auxiliares/modelo_CatBoost.pkl')
    print("Modelo CatBoost guardado como 'Archivos Auxiliares/modelo_CatBoost.pkl'")
# Guardar el modelo
else:
    model = joblib.load('Archivos Auxiliares/modelo_CatBoost.pkl')
    print("Modelo CatBoost cargado desde 'Archivos Auxiliares/modelo_CatBoost.pkl'")

In [ ]:
# Importancia de las características
importancias = model.get_feature_importance()
indices = np.argsort(importancias)[::-1]
print("\nImportancia de las características\n")
for i, idx in enumerate(indices):
    if idx < len(X.columns):
        print(f"{i + 1:2d}. {X.columns[idx]}: {importancias[idx]:.4f}")

In [ ]:
# Eliminar características con importancia menor a 5
umbral_importancia = 5

# Solo considera las columnas usadas por el modelo y presentes en X_train/X_test
columnas_modelo = X.columns[:len(importancias)]
FEATURES = [col for col, imp in zip(columnas_modelo, importancias) 
            if imp >= umbral_importancia and col in X_train.columns and col in X_test.columns]

X_train = X_train[FEATURES]
X_test = X_test[FEATURES]

# Reentrenar el modelo con las características seleccionadas
model.fit(X_train, y_train)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
graph_pred(Y_pred, 'CatBoost_C')

In [ ]:
# Graficar importancia de variables
importances = model.feature_importances_
importance_df = pd.DataFrame({"Feature": FEATURES, "Importance": importances})
importance_df = importance_df.sort_values(by="Importance", ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(importance_df["Feature"], importance_df["Importance"], color='skyblue')
plt.xlabel("Importancia")
plt.ylabel("Feature")
plt.title("CatBoost: Importancia de variables")
plt.gca().invert_yaxis() 
plt.tight_layout()
plt.savefig('CatBoost_importancia.png')

print("\nSe han generado las siguientes visualizaciones:")
print("- CatBoost_predicciones.png (Comparación de valores reales vs. predichos)")
print("- CatBoost_importancia.png (Importancia de las variables)")

# XGBoost